In [1]:
from openai import OpenAI

In [2]:
client = OpenAI()

In [3]:
batch_file = client.files.create(
  file=open("../../eval_data/batch_tasks_question_generation.jsonl", "rb"),
  purpose="batch"
)

In [16]:
print(batch_file)

FileObject(id='file-DqEhjEnFkoJruLAhTphdp2', bytes=92980819, created_at=1750638917, filename='batch_tasks_question_generation.jsonl', object='file', purpose='batch', status='processed', expires_at=None, status_details=None)


In [5]:
batch_job = client.batches.create(
  input_file_id=batch_file.id,
  endpoint="/v1/chat/completions",
  completion_window="24h"
)

In [17]:
batch_job = client.batches.retrieve(batch_job.id)
print(batch_job)

Batch(id='batch_6858a16e8e3c8190bdbfd3d1abb48d63', completion_window='24h', created_at=1750638958, endpoint='/v1/chat/completions', input_file_id='file-DqEhjEnFkoJruLAhTphdp2', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1750639474, error_file_id=None, errors=None, expired_at=None, expires_at=1750725358, failed_at=None, finalizing_at=1750639419, in_progress_at=1750638962, metadata=None, output_file_id='file-Ftjjzx6PGAWWzuXeC2un13', request_counts=BatchRequestCounts(completed=115, failed=0, total=115))


In [18]:
result_file_id = batch_job.output_file_id
result = client.files.content(result_file_id).content

In [19]:
result_file_name = "../../eval_data/batch_job_results_image_eval.jsonl"

with open(result_file_name, 'wb') as file:
    file.write(result)

In [20]:
import json

In [21]:
# Loading data from saved file
results = []
with open(result_file_name, 'r') as file:
    for line in file:
        # Parsing the JSON string into a dict and appending to the list of results
        json_object = json.loads(line.strip())
        results.append(json_object)

In [26]:
sources_file = "../../eval_data/batch_tasks_question_generation_sources.json"

with open(sources_file, "r") as f:
    sources = json.load(f)

In [28]:
results[0]

{'id': 'batch_req_6858a33c7bd881909580f98d51ad2505',
 'custom_id': 'task-0',
 'response': {'status_code': 200,
  'request_id': '06a89d4882fad41bd8bf039b97946cf0',
  'body': {'id': 'chatcmpl-BlPXN6mlC7DjeXRTyhREoC2QOchU7',
   'object': 'chat.completion',
   'created': 1750639281,
   'model': 'gpt-4o-mini-2024-07-18',
   'choices': [{'index': 0,
     'message': {'role': 'assistant',
      'content': '{"questions":[{"input":"What is the course code for the Python for Data Science class?","expected_output":"ECE 20875"},{"input":"Who are the authors of the Python for Data Science material?","expected_output":"Qiang Qiu, Murat Kocaoglu, and Anuran Makur"},{"input":"What topic is covered in the ECE 20875 course?","expected_output":"Probability and Random Variables"}]}',
      'refusal': None,
      'annotations': []},
     'logprobs': None,
     'finish_reason': 'stop'}],
   'usage': {'prompt_tokens': 37075,
    'completion_tokens': 89,
    'total_tokens': 37164,
    'prompt_tokens_details': 

In [37]:
eval_dataset = []

for result in results:
    custom_id: str = result["custom_id"]
    idx = int(custom_id[custom_id.index("-")+1:])
    eval_dataset.extend([{**qa_data, **sources[idx]} for qa_data in json.loads(result["response"]["body"]["choices"][0]["message"]["content"])["questions"]])

In [39]:
with open("../../eval_data/image_eval_data.json", "w") as f:
    json.dump(eval_dataset, f, indent=4)